# Improving Access to Clean, Safe Drinking Water in Tanzania
## Exploratory Data Analysis - Part 2

Kenneth Liao

---

## Background

The UN publishes and reviews a list of least developed countries (LDC) every 3 years. LDCs are “low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.”$^{1}$. Tanzania has been classified as an LDC since the UN published the first list of LDCs in 1971$^{2}$. A common challenge of LDCs is a lack of infrastructure to support the development of the nation, including access to education and healthcare, waste management, and potable water.

According to UNICEF, as of 2017, more than 24 million Tanzanians lacked access to basic drinking water$^{3}$. This corresponds to only 56.7% of the country’s population having access to basic drinking water. Outside of developed urban areas, much of the potable water is accessed via water pumps. 

Taarifa is an open-source platform for crowd-sourced reporting and triaging of infrastructure related issues. Together with the Tanzanian Ministry of Water, data has been collected for thousands of water pumps throughout Tanzania. The goal of this project is to be able to predict the condition of these water pumps to improve maintenance, reduce pump downtime, and ensure basic water access for tens of millions of Tanzanians.

**References**

1. https://www.un.org/development/desa/dpad/least-developed-country-category.html
2. https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/publication/ldc_list.pdf
3. https://washwatch.org/en/countries/tanzania/summary/statistics/


### Problem Description

Predict the operating condition of water pumps in Tanzania given various metadata on each water pump.

### Strategy

The strategy will be to implement a Random Forest model for multiclass classification of the state of water pumps.

### Data

The dataset is provided by Taarifa, together with the Tanzanian Ministry of Water and is hosted by DrivenData.org:

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/

---

## Exploratory Data Analysis

Start by importing the necessary libraries and loading the data.

In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.express as px
from config import credentials

init_notebook_mode(connected=True)

In [2]:
# load the data
train = pd.read_csv('../data/train.csv')
train_labels = pd.read_csv('../data/train-labels.csv')

`amount_tsh` - Total static head (amount water available to waterpoint)
<br>`date_recorded` - The date the row was entered
<br>`funder` - Who funded the well
<br>`gps_height` - Altitude of the well
<br>`installer` - Organization that installed the well
<br>`longitude` - GPS coordinate
<br>`latitude` - GPS coordinate
<br>`wpt_name` - Name of the waterpoint if there is one
<br>`num_private` -
<br>`basin` - Geographic water basin
<br>`subvillage` - Geographic location
<br>`region` - Geographic location
<br>`region_code` - Geographic location (coded)
<br>`district_code` - Geographic location (coded)
<br>`lga` - Geographic location
<br>`ward` - Geographic location
<br>`population` - Population around the well
<br>`public_meeting` - True/False
<br>`recorded_by` - Group entering this row of data
<br>`scheme_management` - Who operates the waterpoint
<br>`scheme_name` - Who operates the waterpoint
<br>`permit` - If the waterpoint is permitted
<br>`construction_year` - Year the waterpoint was constructed
<br>`extraction_type` - The kind of extraction the waterpoint uses
<br>`extraction_type_group` - The kind of extraction the waterpoint uses
<br>`extraction_type_class` - The kind of extraction the waterpoint uses
<br>`management` - How the waterpoint is managed
<br>`management_group` - How the waterpoint is managed
<br>`payment` - What the water costs
<br>`payment_type` - What the water costs
<br>`water_quality` - The quality of the water
<br>`quality_group` - The quality of the water
<br>`quantity` - The quantity of water
<br>`quantity_group` - The quantity of water
<br>`source` - The source of the water
<br>`source_type` - The source of the water
<br>`source_class` - The source of the water
<br>`waterpoint_type` - The kind of waterpoint
<br>`waterpoint_type_group` - The kind of waterpoint

In [10]:
def plot_hist(data, col, ylog=False, xlog=False):
    """
    Convenience function for plotting the histogram
    for a given feature.
    """
    
    if ylog:
        ymode='log'
    else:
        ymode=None
    if xlog:
        xmode='log'
    else:
        xmode=None
    
    trace = go.Histogram(x=data[col], name='col')
    
    layout = go.Layout(title=f'{col} Distribution',
                  yaxis=dict(title='Count', type=ymode),
                       xaxis=dict(type=xmode))
        
    fig = go.Figure([trace], layout=layout)
    
    iplot(fig, filename=f'{col}-dist.html')